In [2]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
import numpy as np

2024-03-30 00:01:03.462540: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-30 00:01:03.504950: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-30 00:01:04.252277: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

In [4]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

2024-03-30 00:01:12.084967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8999 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3b:00.0, compute capability: 7.5
2024-03-30 00:01:12.085694: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9235 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5
2024-03-30 00:01:12.086310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9235 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:d8:00.0, compute capability: 7.5
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transforme

In [170]:
sent = '근육이 커지기 위해서는'

In [171]:
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

tf.Tensor([[33245 10114 12748 11357]], shape=(1, 4), dtype=int32)


In [172]:
#output = model.generate(input_ids,
#                        max_length=128,
#                        repetition_penalty=2.0,
#                        use_cache=True)

output = model(input_ids)
Logits = output.logits
Logits

<tf.Tensor: shape=(1, 4, 51200), dtype=float32, numpy=
array([[[-5.2968388 , -5.2453256 , -4.7947283 , ...,  0.02459761,
         -1.1457479 , -0.649338  ],
        [-4.790766  , -5.4113317 , -4.8759828 , ..., -0.9310436 ,
         -3.6518137 , -5.8186173 ],
        [-4.0485926 , -3.5684729 , -5.736005  , ..., -2.4099336 ,
         -2.463568  , -3.5291796 ],
        [-5.4558268 , -6.0249434 , -6.139882  , ..., -0.22451068,
         -4.026171  , -3.2468688 ]]], dtype=float32)>

In [173]:
Logits.shape

TensorShape([1, 4, 51200])

In [174]:

last_index = input_ids.shape[-1]

while input_ids.shape[-1] < 50:
    print("================================")
    print(input_ids.shape[-1])
    #set 나눠서 green set 로짓을 촉진하기
    gamma = 0.5
    delta = 10.0
    Logits = model(input_ids).logits
    print(f"original Logits : {Logits}")

    #seed = int(hash(input_ids % 51200))
    seed = input_ids[-1] % Logits.shape[-1]
    print(f"seed : {seed[-1]}")
    np.random.seed(seed[-1])

    indices = np.arange(Logits.shape[-1]) #0~51999
    green_list_size = int(Logits.shape[-1] * gamma)  #25600
    green_list = np.random.choice(indices, green_list_size, replace=False)
    print(f"green list : {green_list}") #25600개의 green list

    #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
    # 인덱스와 업데이트를 위한 텐서 생성
    print(last_index)
    indices = tf.constant([[0, last_index-1, idx] for idx in green_list])
    updates = tf.constant([delta] * green_list_size)

    # Logits의 특정 요소를 업데이트
    Logits = tf.tensor_scatter_nd_add(Logits, indices, updates)

    print(f"updated Logits : {Logits}")

    # 데이터 타입 확인 및 조정
    add_token_id = tf.argmax(Logits[0][-1])
    print(f"updated Logits Max value : {add_token_id}")


    # 텐서 연결
    add_token_id = tf.reshape(add_token_id, [1,1])
    add_token_id = tf.cast(add_token_id, input_ids.dtype)
    input_ids = tf.concat([input_ids, add_token_id], axis=-1)
    next_token = tokenizer.decode(input_ids.numpy()[0,:])
    print(f"next token : {next_token}")
    
    last_index = input_ids.shape[-1]

4
original Logits : [[[-5.2968388  -5.2453256  -4.7947283  ...  0.02459761 -1.1457479
   -0.649338  ]
  [-4.790766   -5.4113317  -4.8759828  ... -0.9310436  -3.6518137
   -5.8186173 ]
  [-4.0485926  -3.5684729  -5.736005   ... -2.4099336  -2.463568
   -3.5291796 ]
  [-5.4558268  -6.0249434  -6.139882   ... -0.22451068 -4.026171
   -3.2468688 ]]]
seed : 11357
green list : [16373 16203 46098 ... 22325 28635 22050]
4
updated Logits : [[[-5.2968388  -5.2453256  -4.7947283  ...  0.02459761 -1.1457479
   -0.649338  ]
  [-4.790766   -5.4113317  -4.8759828  ... -0.9310436  -3.6518137
   -5.8186173 ]
  [-4.0485926  -3.5684729  -5.736005   ... -2.4099336  -2.463568
   -3.5291796 ]
  [-5.4558268   3.9750566   3.860118   ...  9.775489   -4.026171
    6.753131  ]]]
updated Logits Max value : 23879
next token : 근육이 커지기 위해서는 무엇보다
5
original Logits : [[[-5.2968383  -5.245327   -4.7947283  ...  0.02459717 -1.1457479
   -0.64933836]
  [-4.790767   -5.411331   -4.8759823  ... -0.9310415  -3.6518152
   -5

In [36]:
#top5 = tf.math.top_k(output.logits[0, -1], k=5)

In [ ]:
output_ids = output.numpy().tolist()[0]
print(output_ids)

In [8]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'